In [27]:
import pandas as pd
import os

# Historical + Current Box Office Data

Sourced from sovai - includes film titles, distributers, total gross and approximate gross per theatre, etc.

In [28]:
def normalize_title(s):
    s = s.astype(str).str.lower().str.strip()
    return s

In [29]:
sov_df = pd.read_csv("../data/raw/sov_data.csv")

In [30]:
sov_df.describe()

,gross,percent_yd,percent_lw,theaters,per_theater,total_gross,days_in_release,year
count,2.363670e+05,236367.000000,236367.000000,236367.000000,2.363670e+05,2.363670e+05,236367.000000,236367.000000
mean,7.920085e+05,0.247164,0.009030,1117.919295,5.242793e+02,5.913805e+07,105.135793,2013.042421
std,2.407195e+06,15.463792,8.101669,1460.888042,3.780200e+03,8.884364e+07,849.949356,6.685246
min,0.000000e+00,-1.000000,-1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,1997.000000
25%,1.081100e+04,-0.330000,-0.530000,83.000000,9.400000e+01,5.419357e+06,15.000000,2009.000000
50%,7.765400e+04,-0.030000,-0.350000,428.000000,2.010000e+02,2.734414e+07,34.000000,2014.000000
75%,5.683805e+05,0.280000,0.000000,2186.000000,4.590000e+02,7.460862e+07,63.000000,2018.000000
max,1.219647e+08,6265.000000,1719.000000,380673.000000,1.695119e+06,8.148069e+08,30725.000000,2025.000000


In [31]:
sov_df['title_key'] = normalize_title(sov_df['title'])

In [ ]:
# Clean SOVAI data
# Drop duplicates: keep latest/highest grossing per movie
sov_df = sov_df.sort_values(
    by=['title_key', 'date', 'total_gross', 'theaters'],
    ascending=[True, False, False, False]
)
sov_latest = sov_df.drop_duplicates(subset=['title_key'], keep='first').copy()

# Convert numeric columns
for col in ['gross', 'per_theater', 'theaters', 'total_gross', 'days_in_release']:
    if col in sov_latest.columns:
        sov_latest[col] = pd.to_numeric(sov_latest[col], errors='coerce')

# Drop rows missing critical data
sov_latest = sov_latest.dropna(subset=['title_key', 'gross', 'date'])

# Film Ratings + Other Film Metadata

Sourced from TMDB (the movie database)

Includes aggregate runtimes, popularity ratings, descriptions of movies, number of raters, also includes revenue etc.

[Movie details for a given movie id](https://developer.themoviedb.org/reference/movie-details)


Here is the drive folder with the bulk movie id data:
[download tmdb_movie_ids.csv](https://drive.google.com/file/d/1gOMNDu7MLIriftb3audXMP0UEmZAiHQt/view?usp=sharing)


In [5]:
RAW_DATA_PATH = "raw"

In [7]:
tmdb_df = pd.read_csv(f"../data/{RAW_DATA_PATH}/tmdb_movie_ids.csv")
print(tmdb_df.columns)

Index(['adult', 'id', 'original_title', 'popularity', 'video'], dtype='object')


In [32]:
tmdb_df['title_key'] = normalize_title(tmdb_df['original_title'])

In [33]:
tmdb_df.sort_values(by='popularity', ascending=False)

,id,original_title,popularity,title_key
811727,1156594,Culpa nuestra,532.1524,culpa nuestra
1073852,1511789,Captain Hook - The Cursed Tides,340.5134,captain hook - the cursed tides
534839,755898,War of the Worlds,317.3022,war of the worlds
907504,1280450,Stolen Girl,304.1407,stolen girl
926600,1305717,Hunting Grounds,300.4624,hunting grounds
...,...,...,...,...
807723,1151169,Fade Away,0.0000,fade away
807749,1151216,Scenic National Parks: Zion and Bryce,0.0000,scenic national parks: zion and bryce
807758,1151233,Pompeii: The Doomed City,0.0000,pompeii: the doomed city
807760,1151235,"When It Rayns, It Pours",0.0000,"when it rayns, it pours"


# DATA CLEANING TO MERGE



1.   Normalizing Titles: Movie titles often contain variations in capitalization, spacing, punctuation, etc., which can prevent matching across different datasets. To address this, we normalize all titles to the same format, creating a consistent key for merging called title_key.
2. SOVAI Data Cleaning: This dataset contains historical performance data, including gross revenue, number of theaters, and release dates. To clean up this data:

*   Duplicate entries are removed, keeping the row with the latest release data and highest revenue
*   Numeric columns (gross, total_gross, theaters) are converted to numeric types
* Rows missing critical data (title_key, gross, date) are dropped
3. TMDB Data Cleaning: This dataset contains basic metadata including ID, title, popularity – which we keep and normalize as needed, also converting popularity to a numeric value
4. Merging Datasets: The cleaned datasets are merged on title_key, ensuring that only movies present in both datasets are included and thus also combining data from both sources.
5. Feature Engineering/Extraction: Additional features can be derived from the newly created dataset to improve predictive power. One example  includes a flag indicating if the date released is_weekend.


In [34]:
# Clean TMDB CSV (minimal)
# Keep only relevant columns
tmdb_df = tmdb_df[['id', 'original_title', 'popularity', 'title_key']].copy()
tmdb_df['popularity'] = pd.to_numeric(tmdb_df['popularity'], errors='coerce')

# Merge

In [38]:

# Merge SOVAI + TMDB
merged_df = pd.merge(
    sov_latest,
    tmdb_df,
    how='inner',  # only keep movies present in both
    on='title_key',
    suffixes=('_sov', '_tmdb')
)

# Feature engineering - Weekday/weekend flag based on SOVAI date
merged_df['weekday'] = pd.to_datetime(merged_df['date']).dt.weekday
merged_df['release_month'] = pd.to_datetime(merged_df['release_date']).dt.month
merged_df['release_weekday'] = pd.to_datetime(merged_df['release_date']).dt.dayofweek
merged_df['is_weekend'] = merged_df['weekday'].isin([4,5,6]).astype(int)

n=30
merged_df[['title_key', 'date', 'gross', 'popularity', 'is_weekend']].head(n) #printing first n


,title_key,date,gross,popularity,is_weekend
0,10 cloverfield lane,2016-06-02,11414,7.8372,0
1,100 meters,2025-10-16,313,0.0143,0
2,102 not out,2018-06-03,2806,3.4544,1
3,10th & wolf,2006-09-04,1791,3.6557,0
4,11th hour,2007-09-03,64888,2.3114,0
5,11th hour,2007-09-03,64888,0.1843,0
6,12,2009-05-25,344,2.6577,0
7,12,2009-05-25,344,0.2050,0
8,12,2009-05-25,344,0.2622,0
9,12,2009-05-25,344,1.3215,0


In [39]:
merged_df.to_csv("../data/cleaned/merged_df.csv", index=False)

In [40]:
import pandas as pd
from tabulate import tabulate

# --- SOV.AI SCHEMA ---
schema_sovai = pd.DataFrame({
    "Data Field": sov_latest.columns,
    "Data Type": sov_latest.dtypes.astype(str)
})

print("SOV.AI Dataset Schema:")
print(tabulate(schema_sovai, headers="keys", tablefmt="github"))
print("\n")


# --- TMDB SCHEMA ---
schema_tmdb = pd.DataFrame({
    "Data Field": tmdb_df.columns,
    "Data Type": tmdb_df.dtypes.astype(str)
})

print("TMDB Dataset Schema:")
print(tabulate(schema_tmdb, headers="keys", tablefmt="github"))
print("\n")


# --- MERGED DATASET SCHEMA ---
schema_merged = pd.DataFrame({
    "Data Field": merged_df.columns,
    "Data Type": merged_df.dtypes.astype(str)
})

print("Merged Dataset Schema:")
print(tabulate(schema_merged, headers="keys", tablefmt="github"))
print("\n")


# OPTIONAL: save to CSV files
schema_sovai.to_csv("schemas/schema_sovai.csv", index=False)
schema_tmdb.to_csv("schemas/schema_tmdb.csv", index=False)
schema_merged.to_csv("schemas/schema_merged.csv", index=False)


SOV.AI Dataset Schema:
|                     | Data Field          | Data Type   |
|---------------------|---------------------|-------------|
| ticker              | ticker              | object      |
| date                | date                | object      |
| title               | title               | object      |
| distributor         | distributor         | object      |
| gross               | gross               | int64       |
| percent_yd          | percent_yd          | float64     |
| percent_lw          | percent_lw          | float64     |
| theaters            | theaters            | float64     |
| per_theater         | per_theater         | float64     |
| total_gross         | total_gross         | int64       |
| days_in_release     | days_in_release     | int64       |
| parent company      | parent company      | object      |
| distributor address | distributor address | object      |
| distributorwebsite  | distributorwebsite  | object      |
| release_date   